In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Для начала вам потребуется загрузить данные. В этом задании мы воспользуемся одним из датасетов, доступных в scikit-learn'е — 20 newsgroups. Для этого нужно воспользоваться модулем datasets:

In [2]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

# 2. После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

In [3]:
X = newsgroups.data
y = newsgroups.target
print("targets: ", y)
print("target_names: ", newsgroups.target_names)

targets:  [0 0 1 ..., 1 1 0]
target_names:  ['alt.atheism', 'sci.space']


# 3. Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.


In [4]:
print("Extracting features from the training data using vectorizer")
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X)
print("n_samples: %d, n_features: %d" % X_train.shape)

Extracting features from the training data using vectorizer
n_samples: 1786, n_features: 28382


# 4. Реализация SVM-классификатора находится в классе sklearn.svm.SVC. Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:

In [5]:
# mapping from integer feature name to original token string
feature_names = np.asarray(vectorizer.get_feature_names())
feature_names

array(['00', '000', '0000', ..., 'zwarte', 'zwork', 'zyklon'], 
      dtype='<U80')

# 5. Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV. Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. 

In [6]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_train, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

# 6. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

In [10]:
df = pd.DataFrame.from_dict(gs.cv_results_)
df.head()
df_with_best_score = df[df['mean_test_score'] == (df['mean_test_score'].max())]
best_C = df_with_best_score['param_C'].max()
best_C

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,2.698479,0.679409,0.552632,0.552632,1e-05,{'C': 1e-05},8,0.544693,0.554622,0.579832,...,0.571429,0.547936,0.501401,0.565430,0.565826,0.549335,0.112911,0.029222,0.028117,0.007027
1,2.602821,0.643558,0.552632,0.552632,0.0001,{'C': 0.0001},8,0.544693,0.554622,0.579832,...,0.571429,0.547936,0.501401,0.565430,0.565826,0.549335,0.026827,0.021861,0.028117,0.007027
2,2.637450,0.624828,0.552632,0.552632,0.001,{'C': 0.001},8,0.544693,0.554622,0.579832,...,0.571429,0.547936,0.501401,0.565430,0.565826,0.549335,0.018705,0.007099,0.028117,0.007027
3,2.547836,0.631150,0.552632,0.552632,0.01,{'C': 0.01},8,0.544693,0.554622,0.579832,...,0.571429,0.547936,0.501401,0.565430,0.565826,0.549335,0.043327,0.008792,0.028117,0.007027
4,2.159612,0.548699,0.950168,0.964305,0.1,{'C': 0.1},7,0.958101,0.960784,0.949580,...,0.957983,0.960812,0.935574,0.961512,0.949580,0.967810,0.036324,0.017626,0.008218,0.004109


100000.0

# 7. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [8]:
clf.set_params(C = best_C).fit(X_train, y)

SVC(C=100000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

# 8. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [9]:
absolute_data = abs(clf.coef_.toarray().reshape(-1)) # by absolute

absolute_data_sorted_desc = sorted(absolute_data, reverse=True)
weight_indexes = []
for weight in absolute_data_sorted_desc[:10]:
    weight_indexes.append(absolute_data.tolist().index(weight))   
        
words = [vectorizer.get_feature_names()[index] for index in weight_indexes]

print('%s' % (" ".join(sorted(words))))

atheism atheists bible god keith moon nick religion sky space
